# Multi-Tab Health Calculator App using Gradio
EGBI121 Mini project by Kritpatchara Wongkwan 6813353 and Wafi Alsawad 6813426


The BME Health Suite is a multi-tab web application, running on a local device, designed to provide users with essential health and fitness calculations. Built in Python and deployed with Gradio, the application serves as a comprehensive tool with three tabs for calculating and tracking Body Mass Index (BMI), calculating Basal Metabolic Rate (BMR), and logging daily food intake.

### Import necessary libraries

In [1]:
%pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: gradio
    Found existing installation: gradio 5.49.0
    Uninstalling gradio-5.49.0:
      Successfully uninstalled gradio-5.49.0


In [2]:
# Import necessary libraries

# Data manipulation
import pandas as pd

# Visualization
import plotly.express as px
import plotly.graph_objects as go

# Date and time
from datetime import datetime

# Data persistence
import os

# Web application interface
import gradio as gr

## Backend development

### Tab 1 function: BMI

Define a function for Body Mass Index (BMI) calculation using the following equation: $$\text{BMI} = \frac{\text{weight}}{(\text{height})^2}$$
where weight is in kilograms and height is in meters.

In [3]:
# BMI calculation function
def calculate_bmi(weight, height):
    # Validate inputs
    if not isinstance(height, (int, float)) or not isinstance(weight, (int, float)) or height <= 0 or weight <= 0:
        return "Please enter the valid value!", None

    # Convert cm to m
    height = height / 100

    bmi = weight / (height ** 2)
    return round(bmi, 2), None

# Note: For BMI calculation, the unit of weight is kg and height is m.

In this application, users can input data in either the metric (kg and cm) or imperial (lbs and in) system. However, since the BMI calculation uses the metric system, a unit conversion function needs to be defined.

In [4]:
def convert_to_Metric(weight, height):
    # Convert lbs and in to kg and cm
    return weight * 0.45, height * 2.54

For the category, this project uses standard BMI classification system from Bidani et al.(2020). There are five classes as follows:

| **BMI**     | **Class**                      |
|--------------|-------------------------------|
| < 18.5       | Underweight                   |
| 18.5–24.9    | Normal                        |
| 25.0–29.9    | Overweight                    |
| 30.0–34.9    | Obesity (1st Class)           |
| 35.0–39.9    | Obesity (2nd Class)           |
| > 40         | Extreme Obesity (3rd Class)   |


In [5]:
# BMI Category function (Bidani et al., 2020)
def find_bmi_category(bmi):
    bmi_category = ""
    if bmi < 18.5:
        bmi_category = "Underweight"
    elif bmi < 24.9:
        bmi_category = "Normal"
    elif bmi < 29.9:
        bmi_category = "Overweight"
    elif bmi < 34.9:
        bmi_category = "Obesity (1st class)"
    elif bmi < 39.9:
        bmi_category = "Obesity (2nd class)"
    elif bmi >= 40:
        bmi_category = "Extreme Obesity (3rd class)"
    else:
        return "There are some condition error!"
    return bmi_category

The BMI data need to be visualized over time, so the calculated BMI history will be saved in a `.csv` file and retrieved using a DataFrame.

In [6]:
# Function for BMI History

import os
import pandas as pd
from datetime import datetime

bmi_data_file = "bmi_data.csv"

# BMI data loading or initialization function
def load_bmi_data():
    if os.path.exists(bmi_data_file):
        df = pd.read_csv(bmi_data_file)
        # Ensure 'Date' column is datetime type for proper plotting
        df['Date'] = pd.to_datetime(df['Date'])
        return df
    else:
        return pd.DataFrame(columns=["Date", "BMI"])

# BMI data saving function
def save_bmi_data(date, bmi):
    df = load_bmi_data()
    new_entry = pd.DataFrame([{"Date": pd.to_datetime(date).date(), "BMI": bmi}])
    new_entry['Date'] = pd.to_datetime(new_entry['Date'])

    # Remove any duplicate entry for the same date
    df = df[df['Date'] != new_entry['Date'].iloc[0]]

    df = pd.concat([df, new_entry], ignore_index=True)
    df = df.sort_values(by="Date")
    df.to_csv(bmi_data_file, index=False)
    return df

In [7]:
# BMI logging Test

# Setup existing log
# df_existing_bmi_test = pd.DataFrame({
#     "Date": ["2025-03-01", "2025-03-02", "2025-03-03"],
#     "BMI": [20.5, 22.3, 25.1]
# })
# df_existing_bmi_test.to_csv(bmi_data_file, index=False)
df_bmi = load_bmi_data()

# Check the saving function
date = "March 7, 2025"
bmi_new_test = 23.4

df_bmi = save_bmi_data(date, bmi_new_test)

print(df_bmi)

        Date   BMI
0 2025-03-07  23.4


/tmp/ipython-input-3950471438.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_entry], ignore_index=True)


For visualization function and any input data, it will be defined later in the Web application interface session.

### Tab 2 function: Daily Metabolic Rate Calculator

For this tab, the necessary data for Basal Metabolic Rate (BMR) calculation—age, gender, height, weight, and activity level (dropdown)—will be collected from users to calculate the Total Daily Energy Expenditure (TDEE).

Among the numerous methods to determine Basal Metabolic Rate (BMR), the **Harris–Benedict equation** is one of the most widely used by healthcare centers and hospitals. Although it is not the most accurate method available today, it remains a practical and accessible approach.  

This project adopts the equation from the research *A Biometric Study of Human Basal Metabolism* (Harris et al., 1918), as shown below.

**For males:**

$$\text{BMR (kcal)} = 66.4730 + 13.7516 \times \text{Weight (kg)} + 5.0033 \times \text{Height (cm)} - 6.7750 \times \text{Age (years)}$$

**For females:**

$$\text{BMR (kcal)} = 665.0955 + 9.5634 \times \text{Weight (kg)} + 1.8496 \times \text{Height (cm)} - 4.6756 \times \text{Age (years)}$$

In [8]:
# BMR Calculation function

def calculate_bmr(age, gender, height, weight):
    # Data Validation
    if age is None or gender is None or height is None or weight is None or not isinstance(age, (int)) or not isinstance(height, (int, float)) or not isinstance(weight, (int, float)) or age <= 0 or height <= 0 or weight <= 0:
        return "Please fill in all the required fields with the valid value!", ""
    elif gender == "Male":
        bmr = 66.4730 + 13.7516*weight + 5.0033*height - 6.7750*age
    else:
        bmr = 665.0955 + 9.5634*weight + 1.8496*height - 4.6756*age

    return round(bmr, 2)


The Total Daily Energy Expenditure (TDEE) varies depending on the level of physical activity performed throughout the day. When activity levels are high, TDEE increases; conversely, when activity levels are low, TDEE decreases. The basic equation for calculating TDEE is shown below:

$$
TDEE = BMR \times PAL
$$

where $PAL$ represents the *Physical Activity Level*, which serves as a multiplier reflecting overall daily activity.

The PAL values are determined based on Physical Activity as follows:
1) 1.2 for Sedentary
2) 1.5 for Lightly Active
3) 1.7 Moderately Active
4) 1.9 for Very Active
5) 2.2 for Extremely Active

Due to the existence of several calculation methods, this project adopts the calculation approach and PAL factor multipliers from [strengthlab360.com](https://strengthlab360.com/).

In [9]:
    # Physical Activity Levels dictionary

    activity_levels_dict = {
        "Sedentary (little to no exercise)": 1.2,
        "Lightly Active (1-3 exercise sessions/week)": 1.375,
        "Moderately Active (3-5 exercise sessions/week)": 1.55,
        "Very Active (5-6 exercise sessions/week)": 1.725,
        "Extremely Active (more than 6 exercise sessions/week)": 1.9
    }

In [10]:
# TDEE calculation function

def calculate_tdee(bmr, activity_level):
    pal = activity_levels_dict[activity_level]
    tdee = bmr * pal

    return round(tdee, 2)


To achieve the desired weight goal, the daily caloric intake should follow these guidelines:

1. **Weight Loss:** $TDEE − 450$  
2. **Maintenance:** $TDEE$  
3. **Weight Gain:** $TDEE + 450$

According to several health recommendations, the appropriate calorie deficit or surplus for healthy weight loss or gain should be within the range of 200–500 calories per day. In this project, 450 calories are being used.


In [11]:
# Users' Goal

goal_list = ["Weight Loss", "Maintenance", "Weight Gain"]

In [12]:
def find_daily_calories(goal, tdee):

    calorie_goal = tdee
    if goal == "Weight Loss":
        calorie_goal -= 450
    elif goal == "Weight Gain":
        calorie_goal += 450

    return round(calorie_goal, 2)

## Web application interface

Try to initialize the main application interface using Gradio with a `gr.themes.Soft()` built-in theme.

In [13]:
# UI for Main application

with gr.Blocks(title = "BME Health Calculator Suite", theme = gr.themes.Soft()) as app:
    gr.Markdown(
        """
        # 🩺 BME Health Calculator Suite
        A comprehensive health tracking application for Biomedical Engineering students.
        """
    )

app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a269dd5229adc0911a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The application includes three tabs:
- Tab 1: BMI
- Tab 2: Metabolic Rate
- Tab 3: Food Tracking

Each tab will be defined separately as a function, using a `gr.Blocks`, and then combined into the main interface above.

### Tab 1 UI: BMI

In [14]:
# BMI data clearing function
def clear_bmi_data():
    """Clears the CSV file and returns empty values to update the UI."""
    empty_df = pd.DataFrame(columns=["Date", "BMI"])
    empty_df.to_csv(bmi_data_file, index=False)
    # Return empty values for all outputs to clear them
    return "", "", "", None

In [15]:
# UI for Tab 1: BMI

def tab1_fn(unit, weight, height, date):
    if unit == "Imperial (lbs and in)":
        weight, height = convert_to_Metric(weight, height)
    elif unit != "Metric (kg and cm)":
        return "Invalid unit!", "", "", None

    bmi, _ = calculate_bmi(weight, height)
    if isinstance(bmi, str):
        return bmi, "", "", None

    category = find_bmi_category(bmi)
    recommendation = {
        "Underweight": "Increase your calorie intake with balanced nutrition.",
        "Normal": "Maintain a healthy lifestyle with regular exercise.",
        "Overweight": "Consider a calorie-controlled diet and regular activity.",
        "Obesity (1st class)": "Consult a healthcare provider for weight management.",
        "Obesity (2nd class)": "Professional medical advice is recommended.",
        "Extreme Obesity (3rd class)": "Seek immediate professional consultation."
    }.get(category, "")
    df = load_bmi_data()
    df = save_bmi_data(date, bmi)
    # plot = df.plot(x="Date", y="BMI", kind="line", marker="o", title="BMI History", ylabel="BMI", xlabel="Date", grid=True).get_figure()
    fig = px.line(
        df,
        x="Date",
        y="BMI",
        title="BMI History",
        markers=True,  # Creates markers on the data points
        labels={'Date': 'Date', 'BMI': 'BMI'} # Sets axis labels
    )

    fig.update_layout(
        xaxis_showgrid=True, # Enables grid for x-axis
        yaxis_showgrid=True  # Enables grid for y-axis
    )

    plot = fig

    return round(bmi, 2), category, recommendation, plot


with gr.Blocks(title="BMI", theme=gr.themes.Soft()) as tab1_bmi:
    gr.Markdown("## BMI Calculator")
    gr.Markdown("Calculate your Body Mass Index (BMI) and track your progress over time.")

    # Helper: clear BMI History
    # BMI data clearing function
    def clear_bmi_data():
        empty_df = pd.DataFrame(columns=["Date", "BMI"])
        empty_df.to_csv(bmi_data_file, index=False)
        # Return empty values for all outputs to clear them
        return "", "", "", None

    # Helper: update unit labels
    def update_unit_bmi(unit):
        if unit == "Metric (kg and cm)":
            return gr.update(label="Height (cm)"), gr.update(label="Weight (kg)")
        else:
            return gr.update(label="Height (in)"), gr.update(label="Weight (lbs)")

    with gr.Row():
        # Input Column
        with gr.Column(scale=1):
            gr.Markdown("### Inputs")
            unit = gr.Radio(["Metric (kg and cm)", "Imperial (lbs and in)"], label="Unit System", value="Metric (kg and cm)", interactive=True)
            height = gr.Number(label="Height (cm)", minimum=0)
            weight = gr.Number(label="Weight (kg)", minimum=0)
            date = gr.DateTime(label="Date", value=datetime.now())
            clear_btn = gr.Button("Clear History")
            calculate_btn = gr.Button("Calculate BMI", variant="primary")

        # Output Column
        with gr.Column(scale=2):
            gr.Markdown("### Results")
            bmi_output = gr.Textbox(label="Your BMI", interactive=False)
            category_output = gr.Textbox(label="BMI Category", interactive=False)
            recommendation_output = gr.Textbox(label="Recommendation", interactive=False, lines=3)
            plot_output = gr.Plot(label="BMI History")

    # Event Listeners
    unit.change(fn=update_unit_bmi, inputs=unit, outputs=[height, weight])
    calculate_btn.click(
        fn=tab1_fn,
        inputs=[unit, weight, height, date],
        outputs=[bmi_output, category_output, recommendation_output, plot_output]
    )

    clear_btn.click(
        fn=clear_bmi_data,
        inputs=None,
        outputs=[bmi_output, category_output, recommendation_output, plot_output]
    )

tab1_bmi.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://882513279569d383a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Tab 2 UI: Metabolic Rate

In [16]:
# UI for Tab 2: Metabolic Rate

def tab2_fn(unit, age, gender, height, weight, activity_level, goal):
    if unit == "Imperial (lbs and in)":
        weight, height = convert_to_Metric(weight, height)
    elif unit != "Metric (kg and cm)":
        return "Invalid unit!", "", "", None

    bmr = calculate_bmr(age, gender, height, weight)
    tdee = calculate_tdee(bmr, activity_level)
    goal = find_daily_calories(goal, tdee)

    return round(bmr, 2), round(tdee, 2), round(goal, 2)

with gr.Blocks(title="BMR and TDEE", theme=gr.themes.Soft()) as tab2:
    gr.Markdown("## Metabolic Rate Calculator")
    gr.Markdown("Calculate your Basal Metabolic Rate (BMR), Total Daily Energy Expenditure (TDEE), and the appropriate daily calorie intake.")

    # Helper: update unit labels
    def update_unit_bmi(unit):
        if unit == "Metric (kg and cm)":
            return gr.update(label="Height (cm)"), gr.update(label="Weight (kg)")
        else:
            return gr.update(label="Height (in)"), gr.update(label="Weight (lbs)")

    with gr.Row():
        # Input Column
        with gr.Column(scale=1):
            gr.Markdown("### Inputs")
            unit = gr.Radio(["Metric (kg and cm)", "Imperial (lbs and in)"], label="Unit System", value="Metric (kg and cm)", interactive=True)
            age = gr.Number(label = "Age (years)", minimum = 0)
            gender = gr.Radio(["Male", "Female"], label="Gender")
            height = gr.Number(label="Height (cm)", minimum=0)
            weight = gr.Number(label="Weight (kg)", minimum=0)
            activity_level = gr.Dropdown(list(activity_levels_dict.keys()), label="Activity Level")
            goal = gr.Dropdown(goal_list, label="Determine your Goal")
            calculate_btn = gr.Button("Calculate calories", variant="primary")

        # Output Column
        with gr.Column(scale=2):
            gr.Markdown("### Results")
            bmr_output = gr.Textbox(label="Basal Metabolic Rate (BMR)", interactive=False)
            tdee_output = gr.Textbox(label="Total Daily Energy Expenditure (TDEE)", interactive=False)
            calorie_goal_output = gr.Textbox(label="Calories Goal", interactive=False)

    # Event Listeners
    unit.change(fn=update_unit_bmi, inputs=unit, outputs=[height, weight])

    calculate_btn.click(
        fn=tab2_fn,
        inputs=[unit, age, gender, height, weight, activity_level, goal],
        outputs=[bmr_output, tdee_output, calorie_goal_output]
    )

tab2.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1cca6ae362a2452bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Tab 3 UI: Food Tracker

In [18]:
# --- Constants and File Handling for Tab 3 ---
FOOD_DATA_FILE = "food_log.csv"

# Pre-set calorie values for common food items
COMMON_FOODS = {
    "Apple": 95,
    "Banana": 105,
    "Chicken Breast (100g)": 165,
    "Brown Rice (1 cup)": 215,
    "Salad with Vinaigrette": 150,
    "Slice of Pizza": 285,
    "Egg": 78
}

# --- Load existing food log ---
def load_food_log():
    if os.path.exists(FOOD_DATA_FILE):
        df = pd.read_csv(FOOD_DATA_FILE)
        df['Date'] = pd.to_datetime(df['Date'])
        return df
    else:
        return pd.DataFrame(columns=["Date", "Food Item", "Calories"])

# --- Process Food Log ---
def process_food_log(selected_date,
                     entry_type,
                     common_food,
                     manual_food,
                     manual_cals,
                     tdee_goal,
                     log_button_trigger):
    food_df = load_food_log()

    # 1) Add new entry
    if log_button_trigger:
        calories = 0
        food_item = ""

        if not selected_date:
            selected_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        if entry_type == "Common Food" and common_food:
            food_item = common_food
            calories = COMMON_FOODS.get(common_food, 0)
        elif entry_type == "Manual Entry" and manual_food and manual_cals is not None and manual_cals >= 0:
            food_item = manual_food
            calories = manual_cals

        if food_item and calories > 0:
            new_entry = pd.DataFrame([{
                'Date': pd.to_datetime(selected_date),
                'Food Item': food_item,
                'Calories': calories
            }])
            food_df = pd.concat([food_df, new_entry], ignore_index=True)
            food_df.to_csv(FOOD_DATA_FILE, index=False)

    # 2) Daily Summary
    daily_log = pd.DataFrame(columns=["Date", "Food Item", "Calories"])
    daily_total_calories = 0

    if selected_date:
        selected_date_dt = pd.to_datetime(selected_date).date()
        if not food_df.empty:
            food_df['Date'] = pd.to_datetime(food_df['Date'])
            daily_log = food_df[food_df['Date'].dt.date == selected_date_dt].copy()
            daily_log['Date'] = daily_log['Date'].dt.strftime("%Y-%m-%d %H:%M:%S")
            daily_total_calories = daily_log['Calories'].sum()

    total_calories_text = f"{daily_total_calories:.0f} kcal"

    # 3) Weekly Progress Chart
    plot_fig = go.Figure()
    plot_fig.update_layout(
        title="No data available for this week",
        xaxis_title="Day of the Week",
        yaxis_title="Total Calories (kcal)",
        template="plotly_white"
    )

    if selected_date:
        selected_ts = pd.to_datetime(selected_date)
        start_of_week = selected_ts - pd.to_timedelta(selected_ts.dayofweek, unit='d')
        end_of_week = start_of_week + pd.to_timedelta(6, unit='d')

        if not food_df.empty:
            weekly_log = food_df[(food_df['Date'] >= start_of_week) & (food_df['Date'] <= end_of_week)].copy()
        else:
            weekly_log = pd.DataFrame(columns=food_df.columns)

        if not weekly_log.empty:
            weekly_log['DateOnly'] = weekly_log['Date'].dt.date
            weekly_summary = weekly_log.groupby('DateOnly')['Calories'].sum().reset_index()
            weekly_summary['DayName'] = pd.to_datetime(weekly_summary['DateOnly']).dt.day_name()

            plot_fig = px.bar(
                weekly_summary,
                x='DayName',
                y='Calories',
                title=f"Weekly Calorie Intake (Week of {start_of_week.strftime('%Y-%m-%d')})",
                labels={'DayName': 'Day of the Week', 'Calories': 'Total Calories (kcal)'}
            )

            if tdee_goal is not None and tdee_goal > 0:
                plot_fig.add_hline(
                    y=tdee_goal,
                    line_dash="dash",
                    annotation_text="TDEE Goal",
                    annotation_position="bottom right",
                    line_color="red"
                )

            plot_fig.update_layout(
                xaxis={'categoryorder': 'array',
                       'categoryarray': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']}
            )

    # 4) Reset manual fields
    if log_button_trigger:
        reset_manual_food = gr.update(value="")
        reset_manual_cal = gr.update(value=None)
    else:
        reset_manual_food = gr.update()
        reset_manual_cal = gr.update()

    return daily_log, total_calories_text, plot_fig, reset_manual_food, reset_manual_cal


# --- UI for Food Tracker ---
with gr.Blocks(theme=gr.themes.Soft()) as tab3_food_tracker:
    gr.Markdown("## Food Tracker")
    gr.Markdown("Log your daily food intake and track your calorie consumption.")

    with gr.Row():
        # --- Input Column ---
        with gr.Column(scale=1):
            gr.Markdown("### Log Your Meal")
            date_input = gr.DateTime(label="Select Date", value=datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            tdee_input = gr.Number(label="Your TDEE Goal (from Tab 2)", minimum=0, value=0)
            entry_type_radio = gr.Radio(["Common Food", "Manual Entry"], label="Entry Type", value="Common Food")

            with gr.Group() as common_food_group:
                common_food_dropdown = gr.Dropdown(list(COMMON_FOODS.keys()), label="Select a Food")
            with gr.Group(visible=False) as manual_entry_group:
                manual_food_input = gr.Textbox(label="Food Name")
                manual_calorie_input = gr.Number(label="Calories (kcal)", minimum=0)
            log_food_button = gr.Button("Log Food", variant="primary")

        # --- Output Column ---
        with gr.Column(scale=2):
            gr.Markdown("### Daily Summary")
            food_log_output = gr.Dataframe(
                label="Today's Food Log",
                headers=["Date", "Food Item", "Calories"],
                datatype=["str", "str", "number"]
            )
            total_calories_output = gr.Textbox(label="Running Daily Total", interactive=False)
            gr.Markdown("### Weekly Progress")
            plot_output = gr.Plot(label="Weekly Calorie Intake")

    # --- Event Listeners ---
    def toggle_entry_fields(entry_type):
        if entry_type == "Common Food":
            return gr.update(visible=True), gr.update(visible=False)
        else:
            return gr.update(visible=False), gr.update(visible=True)

    entry_type_radio.change(
        fn=toggle_entry_fields,
        inputs=entry_type_radio,
        outputs=[common_food_group, manual_entry_group]
    )

    all_inputs = [date_input, entry_type_radio, common_food_dropdown, manual_food_input, manual_calorie_input, tdee_input]
    all_outputs = [food_log_output, total_calories_output, plot_output, manual_food_input, manual_calorie_input]

    log_food_button.click(
        fn=process_food_log,
        inputs=all_inputs + [gr.State(True)],
        outputs=all_outputs
    )

    date_input.change(
        fn=process_food_log,
        inputs=all_inputs + [gr.State(False)],
        outputs=all_outputs
    )

    tab3_food_tracker.load(
        fn=process_food_log,
        inputs=all_inputs + [gr.State(False)],
        outputs=all_outputs
    )

# --- Launch for testing ---

tab3_food_tracker.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://36a44ae88b7b708a9a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## References

Bidani, S., R, P., Vijayarajan, V., & Prasath, S. (2020). Automatic body mass index detection using correlation of face visual cues. Technology and health care: official journal of the European Society for Engineering and Medicine, 28, 107-112. https://doi.org/10.3233/THC-191850

Harris, J. A., & Benedict, F. G. (1918). A Biometric Study of Human Basal Metabolism. Proc Natl Acad Sci U S A, 4(12), 370-373. https://doi.org/10.1073/pnas.4.12.370
